In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# 제품 데이터 저장 리스트
products_data = []

# Selenium 드라이버 설정
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    # Selenium 탐지 우회
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# 제품 정보 추출 함수
def extract_product_info(product, driver):
    try:
        if product.get("id", "").startswith("adPointProductItem") or "adSmartAreaTop" in product.get("id", ""):
            return False

        photo_elem = product.select_one(".thumb_image img")
        if photo_elem:
            photo = photo_elem.get("data-original") or photo_elem.get("src") or "N/A"
            if "noImg_130.gif" in photo:
                photo = "N/A"
        else:
            photo = "N/A"

        name_elem = product.select_one(".prod_name a")
        name = name_elem.text.strip() if name_elem else "Unknown"
        detail_url = name_elem["href"] if name_elem and "href" in name_elem.attrs else None

        # 상세 페이지에서 spec 수집
        spec_dict = {}
        if detail_url:
            # 현재 탭의 핸들 저장
            main_window = driver.current_window_handle

            # 새 탭 열기 (JavaScript 실행)
            driver.execute_script("window.open('', '_blank');")

            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # 새 탭에서 상세 페이지 열기
            driver.get(detail_url)
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_spec table"))
                )
            except:
                print(f"❗️스펙 테이블 로딩 실패 (대기 시간 초과): {name}")

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")

            spec_table = detail_soup.select_one(".prod_spec .spec_tbl tbody") or detail_soup.select_one(".prod_spec table")
            if spec_table:
                for row in spec_table.find_all("tr"):
                    keys = row.find_all("th", class_="tit")
                    values = row.find_all("td", class_="dsc")
                    for key_elem, value_elem in zip(keys, values):
                        key = key_elem.get_text(strip=True)
                        value = value_elem.get_text(strip=True)
                        spec_dict[key] = value

            # 탭 닫기
            driver.close()

            # 원래 탭으로 돌아가기
            driver.switch_to.window(main_window)

        price_elem = product.select_one(".price_sect a")
        price = price_elem.text.strip().replace(",", "") if price_elem else "N/A"

        rating_elem = product.select_one(".star-single .text__score") or product.select_one(".point_num")
        rating = rating_elem.text.strip() if rating_elem else "N/A"

        product_info = {
            "url": photo,
            "Name": name,
            "Price": price,
            "Rating": rating,
            **spec_dict
        }
        products_data.append(product_info)
        print(f"수집: {name}, 사진: {photo}, 가격: {price}, 평점: {rating}")
        return True

    except Exception as e:
        print(f"제품 정보 수집 오류: {e}")

        # 혹시 새 탭이 열려있는 상태라면 닫고 메인 탭으로 돌아가기
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass

        return False

# 페이지 스크레이핑 함수
def scrape_page(driver, page_num):
    try:
        # 현재 페이지의 제품 정보 수집
        soup = BeautifulSoup(driver.page_source, "html.parser")
        products = soup.select(".prod_item.prod_layer")
        print(f"페이지 {page_num}에서 찾은 제품 수: {len(products)}")

        if not products:
            print("더 이상 제품이 없습니다.")
            return False

        for product in products:
            if product.find_parent(id="adPointArea") or product.find_parent(id="adSmartAreaTop"):
                continue
            extract_product_info(product, driver)

        return True
    except Exception as e:
        print(f"페이지 수집 오류 (페이지 {page_num}): {e}")
        return False

# 크롤링 함수
def crawl_danawa_cpu_to_dataframe(max_pages=None):
    driver = setup_driver()
    try:
        base_url = "https://prod.danawa.com/list/?cate=112747"
        driver.get(base_url)
        time.sleep(3)  # 초기 페이지 로딩 대기

        page_num = 1
        while True:
            if max_pages and page_num > max_pages:
                break

            print(f"페이지 {page_num} 수집 중...")

            # 현재 페이지 수집
            scrape_page(driver, page_num)

            if max_pages and page_num >= max_pages:
                break

            # 다음 페이지로 이동
            try:
                # 페이지 이동 방식 결정
                if page_num % 10 == 0:  # 10, 20, 30 등 10의 배수 페이지일 경우
                    # 다음 페이지 그룹 버튼 클릭
                    next_page_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.edge_nav.nav_next"))
                    )
                    next_page_button.click()
                else:
                    # 일반 페이지 이동
                    driver.execute_script(f"movePage({page_num + 1}); return false;")

                # 페이지 로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_item.prod_layer"))
                )
                time.sleep(2)  # 추가 대기 시간

                page_num += 1
            except Exception as e:
                print(f"다음 페이지로 이동 실패: {e}")
                break

        df = pd.DataFrame(products_data)
        print(f"총 {len(products_data)}개 CPU 제품 수집 완료")
        return df

    finally:
        driver.quit()
        print("Selenium 드라이버 종료")

# 실행
max_pages = 18  # 테스트를 위해 작은 값으로 설정
df = crawl_danawa_cpu_to_dataframe(max_pages=max_pages)
print("\n수집된 데이터프레임 미리보기:")
print(df.head())
df.to_csv("danawa_cpu_products.csv", index=False, encoding="utf-8-sig")
print("데이터가 'danawa_cpu_products.csv' 파일로 저장되었습니다.")

In [16]:
    import pandas as pd
    df = pd.read_csv("danawa_cpu_products.csv", encoding="UTF-8")
    df

,url,Name,Price,Rating,제조회사,등록년월,AMD CPU종류,소켓 구분,코어 수,스레드 수,...,인텔 XTU,인텔 딥러닝부스트,StoreMI,VR Ready 프리미엄,SenseMI,인텔 퀵싱크,AMD Ryzen AI,옵테인,유통회사,AMD PRO
0,//img.danawa.com/prod_img/500000/934/627/img/1...,AMD 라이젠7-5세대 7800X3D (라파엘),511000원,4.9,AMD(제조사 웹사이트 바로가기),2023년 04월,라이젠7-5세대,AMD(소켓AM5),8코어,16스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,//img.danawa.com/prod_img/500000/547/531/img/7...,AMD 라이젠7-6세대 9800X3D (그래니트 릿지),743950원,4.9,AMD(제조사 웹사이트 바로가기),2024년 11월,라이젠7-6세대,AMD(소켓AM5),8코어,16스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,//img.danawa.com/prod_img/500000/687/059/img/6...,인텔 코어 울트라7 시리즈2 265K (애로우레이크),539980원,4.8,인텔(제조사 웹사이트 바로가기),2024년 10월,NaN,인텔(소켓1851),P8+E12코어,20스레드,...,○,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,//img.danawa.com/prod_img/500000/211/741/img/1...,AMD 라이젠5-4세대 5600 (버미어),125950원,4.8,AMD(제조사 웹사이트 바로가기),2022년 04월,라이젠5-4세대,AMD(소켓AM4),6코어,12스레드,...,NaN,NaN,○,○,NaN,NaN,NaN,NaN,NaN,NaN
4,//img.danawa.com/prod_img/500000/499/694/img/2...,AMD 라이젠5-5세대 7500F (라파엘),215000원,4.9,AMD(제조사 웹사이트 바로가기),2023년 07월,라이젠5-5세대,AMD(소켓AM5),6코어,12스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,//img.danawa.com/prod_img/500000/732/099/img/2...,인텔 펜티엄 G2030 (아이비브릿지),5400원,4.6,인텔 펜티엄,2013년 06월,NaN,인텔(소켓1155),듀얼 코어,2스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,//img.danawa.com/prod_img/500000/902/917/img/1...,인텔 셀러론 G1610 (아이비브릿지),6780원,4.1,인텔 셀러론,2013년 02월,NaN,인텔(소켓1155),듀얼 코어,2스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,//img.danawa.com/prod_img/500000/690/824/img/1...,인텔 펜티엄 G645 (샌디브릿지),16250원,4.5,인텔 펜티엄,2012년 10월,NaN,인텔(소켓1155),듀얼 코어,2스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
532,//img.danawa.com/prod_img/500000/439/513/img/1...,인텔 펜티엄 G860 (샌디브릿지),2090원,4.7,인텔 펜티엄,2011년 10월,NaN,인텔(소켓1155),듀얼 코어,2스레드,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공인 대리점,NaN


In [17]:
# c 컬럼의 인덱스 찾기
index = df.columns.get_loc('적합성평가인증')

# c 컬럼 좌측의 컬럼들 선택
left_columns = df.columns[:index]
df = df[left_columns]

# 공백을 기준으로 첫 번째 단어 추출
df['등록년'] = df['등록년월'].str.split().str[0]
df['등록월'] = df['등록년월'].str.split().str[1]
df['제조회사'] = df['제조회사'].str.split('\(').str[0]

df.drop(['등록년월', 'Unnamed: 15'], axis=1, inplace=True)

df

<>:11: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\('
C:\Users\ms0\AppData\Local\Temp\ipykernel_10596\2303062987.py:11: SyntaxWarning: invalid escape sequence '\('
  df['제조회사'] = df['제조회사'].str.split('\(').str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_10596\2303062987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['등록년'] = df['등록년월'].str.split().str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_10596\2303062987.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,url,Name,Price,Rating,제조회사,AMD CPU종류,소켓 구분,코어 수,스레드 수,메모리 규격,...,GPU 코어 속도,SMT(하이퍼스레딩),AMD Ryzen Master,AMD 3D V캐시,패키지 형태,쿨러,시네벤치R23(싱글),시네벤치R23(멀티),등록년,등록월
0,//img.danawa.com/prod_img/500000/934/627/img/1...,AMD 라이젠7-5세대 7800X3D (라파엘),511000원,4.9,AMD,라이젠7-5세대,AMD(소켓AM5),8코어,16스레드,DDR5,...,"2,200MHz",○,○,○,멀티팩(정품),미포함,1788.0,18208.0,2023년,04월
1,//img.danawa.com/prod_img/500000/547/531/img/7...,AMD 라이젠7-6세대 9800X3D (그래니트 릿지),743950원,4.9,AMD,라이젠7-6세대,AMD(소켓AM5),8코어,16스레드,DDR5,...,"2,200MHz",○,○,○,멀티팩(정품),미포함,2073.0,23334.0,2024년,11월
2,//img.danawa.com/prod_img/500000/687/059/img/6...,인텔 코어 울트라7 시리즈2 265K (애로우레이크),539980원,4.8,인텔,NaN,인텔(소켓1851),P8+E12코어,20스레드,DDR5,...,NaN,NaN,NaN,NaN,정품,미포함,2304.0,36309.0,2024년,10월
3,//img.danawa.com/prod_img/500000/211/741/img/1...,AMD 라이젠5-4세대 5600 (버미어),125950원,4.8,AMD,라이젠5-4세대,AMD(소켓AM4),6코어,12스레드,DDR4,...,NaN,○,○,NaN,멀티팩(정품),Wraith Stealth 포함,1460.0,10906.0,2022년,04월
4,//img.danawa.com/prod_img/500000/499/694/img/2...,AMD 라이젠5-5세대 7500F (라파엘),215000원,4.9,AMD,라이젠5-5세대,AMD(소켓AM5),6코어,12스레드,DDR5,...,NaN,○,NaN,NaN,멀티팩(정품),Wraith Stealth 포함,1824.0,13824.0,2023년,07월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,//img.danawa.com/prod_img/500000/732/099/img/2...,인텔 펜티엄 G2030 (아이비브릿지),5400원,4.6,인텔 펜티엄,NaN,인텔(소켓1155),듀얼 코어,2스레드,DDR3,...,"1,050MHz",NaN,NaN,NaN,중고,NaN,NaN,NaN,2013년,06월
530,//img.danawa.com/prod_img/500000/902/917/img/1...,인텔 셀러론 G1610 (아이비브릿지),6780원,4.1,인텔 셀러론,NaN,인텔(소켓1155),듀얼 코어,2스레드,DDR3,...,"1,050MHz",NaN,NaN,NaN,중고,NaN,NaN,NaN,2013년,02월
531,//img.danawa.com/prod_img/500000/690/824/img/1...,인텔 펜티엄 G645 (샌디브릿지),16250원,4.5,인텔 펜티엄,NaN,인텔(소켓1155),듀얼 코어,2스레드,DDR3,...,850MHz,NaN,NaN,NaN,중고,NaN,NaN,NaN,2012년,10월
532,//img.danawa.com/prod_img/500000/439/513/img/1...,인텔 펜티엄 G860 (샌디브릿지),2090원,4.7,인텔 펜티엄,NaN,인텔(소켓1155),듀얼 코어,2스레드,DDR3,...,850MHz,NaN,NaN,NaN,중고,NaN,NaN,NaN,2011년,10월


In [4]:
df.to_csv("cpu.csv", index=False, encoding="utf-8-sig")

In [6]:
df.head().to_csv("cpu_sample.csv", index=False, encoding="utf-8-sig")

In [28]:
# 1. 첫 번째 테이블: idx (1000부터 시작)와 a, b 컬럼
cpu_product_df = pd.DataFrame({
    'product_idx': range(1, len(df) + 1),
    'name': df['Name'],
    'brand': df['제조회사'],
    'price': df['Price'],
    'rating': df['Rating'],
    'category': "CPU"
})

# 2. 두 번째 테이블: idx (index+1), table1의 idx를 외래키로, c 컬럼
cpu_img_df = pd.DataFrame({
    'product_image_idx': range(1, len(df) + 1),
    'product_idx': cpu_product_df['product_idx'],  # table1의 idx를 외래키로
    'img_url': df['url']
})

used_columns = ['Name', 'Price', 'url', 'Rating', '제조회사']
# 원래 데이터프레임에서 사용되지 않은 컬럼 선택
remaining_columns = [col for col in df.columns if col not in used_columns]

# table3 생성
cpu_detail_df = pd.DataFrame({
    'cpu_idx': range(1, len(df) + 1),
    'product_idx': cpu_product_df['product_idx']
})
# 나머지 컬럼 추가
for col in remaining_columns:
    cpu_detail_df[col] = df[col]

In [29]:
cpu_img_df.to_csv("cpu_img.csv", index=False, encoding="utf-8-sig")

In [30]:
cpu_product_df.to_csv("cpu_product.csv", index=False, encoding="utf-8-sig")

In [36]:
cpu_detail_df.to_csv("cpu_detail.csv", index=False, encoding="utf-8-sig")

In [34]:
cpu_detail_df.drop(['세대 구분', '출시일', 'GPU 모델명', 'GPU 코어 속도', 'SMT(하이퍼스레딩)','AMD Ryzen Master', 'AMD 3D V캐시','시네벤치R23(싱글)','시네벤치R23(멀티)'], axis=1, inplace=True)

In [37]:
list(cpu_detail_df)

['cpu_idx',
 'product_idx',
 'AMD CPU종류',
 '소켓 구분',
 '코어 수',
 '스레드 수',
 '메모리 규격',
 '내장그래픽',
 '제조 공정',
 '기본 클럭',
 '최대 클럭',
 'L2 캐시',
 'L3 캐시',
 '연산 체계',
 'TDP',
 'PPT',
 'PCIe 버전',
 '최대 PCIe 레인수',
 '최대 메모리 크기',
 '메모리 클럭',
 '메모리 채널',
 '패키지 형태',
 '쿨러',
 '등록년',
 '등록월']